# Basic Examples

## 01-Introductory example

In [1]:
import simpy

# First example
def example(env):
    value = yield env.timeout(1, value=42)
    print('now=%d, value=%d' % (env.now, value))

env = simpy.Environment()
p = env.process(example(env))
env.run()

now=1, value=42


## 02-Python class example

In [2]:
import simpy

# Second example
class Example():

    def __init__(self, env, delay=10):
        self.env = env
        self.delay = delay

    def process(self):
        print('Before timeout: now=%d' % (env.now))
        value = yield self.env.timeout(self.delay, value=42)
        print('After timeout: now=%d, value=%d' % (env.now, value))


env = simpy.Environment()
e = Example(env,delay=10)
env.process(e.process())
env.run()

Before timeout: now=0
After timeout: now=10, value=42


## 03-Vehicle example

In [3]:
import simpy
import numpy as np

class Vehicle:
    def __init__(self, env, min_speed=80, max_speed=100, fuel_consumption=10, fuel_capacity=35):
        self.env = env
        self.min_speed = min_speed # km/hs
        self.max_speed = max_speed # km/hs
        self.fuel_consumption = fuel_consumption # km/lt
        self.fuel_capacity = fuel_capacity # lt
        self.fuel = fuel_capacity # lt


    def drive(self):
        while True:
            print(f"Start Driving at {self.env.now}")
            travel_time = (self.fuel*self.fuel_consumption)/(np.random.randint(self.min_speed,self.max_speed))
            yield self.env.timeout(travel_time)
            print(f"Need refueling at {self.env.now}")

            print(f"Start refueling at {self.env.now}")
            yield self.env.timeout(np.random.uniform(0.05, 0.15))
            self.fuel = np.random.randint(self.fuel_capacity-5,self.fuel_capacity)
            print(f"Finished refueling at {self.env.now}: fuel now is {self.fuel}")


env = simpy.Environment()
v = Vehicle(env)
env.process(v.drive())
env.run(until=24)

Start Driving at 0
Need refueling at 3.977272727272727
Start refueling at 3.977272727272727
Finished refueling at 4.110372414993065: fuel now is 34
Start Driving at 4.110372414993065
Need refueling at 7.579760170095105
Start refueling at 7.579760170095105
Finished refueling at 7.644977975700776: fuel now is 32
Start Driving at 7.644977975700776
Need refueling at 11.049233294849712
Start refueling at 11.049233294849712
Finished refueling at 11.176646696707877: fuel now is 31
Start Driving at 11.176646696707877
Need refueling at 14.37252298536767
Start refueling at 14.37252298536767
Finished refueling at 14.47840208630337: fuel now is 31
Start Driving at 14.47840208630337
Need refueling at 18.30556258013053
Start refueling at 18.30556258013053
Finished refueling at 18.43592311018481: fuel now is 30
Start Driving at 18.43592311018481
Need refueling at 21.627412471886938
Start refueling at 21.627412471886938
Finished refueling at 21.68003469993764: fuel now is 31
Start Driving at 21.680034

## 04-Machine Example

In [4]:
# Example with an entity being processed
import copy

class Product():
    def __init__(self, id):
        self.id = id
        self.processed = False


class Machine():
    def __init__(self, env:simpy.Environment, id, process_time=(10,15,20)):
        self.env = env
        self.id = id
        self.process_time = process_time
        self.idle = True 

    def generate_process_time(self):
        return np.random.triangular(self.process_time[0],self.process_time[1],self.process_time[2])

    def process_product(self, product):
        print(f"Processing product {product.id} at time {self.env.now}")
        self.idle = False
        yield self.env.timeout(delay=self.generate_process_time())
        product.processed = True
        self.idle = False
        return product

    def _set_production_plan(self, products):
        self.pending_products = copy.deepcopy(products)
        self.processed_products = []

    def process_all_products_randomly(self,products):
        self._set_production_plan(products)
        while len(self.pending_products)>0:
            next_prod = np.random.choice(range(len(self.pending_products)))
            p = self.pending_products[next_prod]
            yield env.process(m.process_product(p))
            self.pending_products.remove(p)
            self.processed_products.append(p)

    def process_all_products_fifo(self,products):
        self._set_production_plan(products)
        while len(self.pending_products)>0:
            p = self.pending_products[0]
            yield env.process(m.process_product(p))
            self.pending_products.remove(p)
            self.processed_products.append(p)

    def process_all_products_lifo(self,products):
        self._set_production_plan(products)
        while len(self.pending_products)>0:
            p = self.pending_products[-1]
            yield env.process(m.process_product(p))
            self.pending_products.remove(p)
            self.processed_products.append(p)



print("Processing RANDOM policy")
env = simpy.Environment()
m = Machine(env,id='machine')
products = [Product(id=i) for i in range(10)]
env.process(m.process_all_products_randomly(products))
env.run()
print()

print("Processing FIFO policy")
env = simpy.Environment()
m = Machine(env,id='machine')
products = [Product(id=i) for i in range(10)]
env.process(m.process_all_products_fifo(products))
env.run()
print()

print("Processing LIFO policy")
env = simpy.Environment()
m = Machine(env,id='machine')
products = [Product(id=i) for i in range(10)]
env.process(m.process_all_products_lifo(products))
env.run()

Processing RANDOM policy
Processing product 5 at time 0
Processing product 6 at time 12.146334706420241
Processing product 9 at time 25.41922320679302
Processing product 4 at time 40.41212099773223
Processing product 1 at time 56.588785123494546
Processing product 3 at time 72.90007562837951
Processing product 0 at time 86.80018851221942
Processing product 2 at time 100.05120682763642
Processing product 7 at time 118.10001346524449
Processing product 8 at time 132.93144246633773

Processing FIFO policy
Processing product 0 at time 0
Processing product 1 at time 16.536097773367587
Processing product 2 at time 33.489174435881715
Processing product 3 at time 46.470758671131634
Processing product 4 at time 61.69361932247431
Processing product 5 at time 79.3962876115119
Processing product 6 at time 97.03127374365525
Processing product 7 at time 110.8215196891729
Processing product 8 at time 126.91600354007943
Processing product 9 at time 145.3705580020133

Processing LIFO policy
Processing 